In [ ]:
# ระบบผู้ช่วยทางการแพทย์อัจฉริยะสำหรับวิเคราะห์อาการเบื้องต้นและให้คำแนะนำทางการแพทย์ภาษาไทย พร้อม RAG
# บริษัท วี89 เทคโนโลยี จำกัด
# Version 2.0 Thai Medical Care AI Agent Chat RAG system with Multi-Modal in Google Colab T4 GPU

print("🚀 Installing required packages...")

# Install required packages with proper versions
!pip install -q --upgrade pip==24.0
!pip install -q --force-reinstall torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu124
!pip install -q compressed-tensors>=0.7.0
!pip install -q transformers==4.50.0 accelerate==1.1.0 bitsandbytes==0.47.0
!pip install -q numpy==2.0.2 fsspec==2025.3.2 jedi>=0.16 filelock>=3.15 typing_extensions>=4.14.0  filelock>=3.15 scipy==1.14.1 websockets==15.0.1
!pip install -q pythainlp sentence-transformers
!pip install -q datasets==2.20.0 evaluate rouge-score==0.1.2
!pip install -q chromadb>=0.4.0 faiss-cpu
!pip install -q pandas scikit-learn
!pip install -q openai-whisper timm Pillow
!pip install -q gradio>=4.44.1
!pip install -q opencv-python-headless>=4.9.0.8 soundfile librosa
!pip install -q optuna

print("📦 Packages installed successfully!")

import torch
import gradio as gr
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    ViTImageProcessor,
    ViTForImageClassification
)
from datetime import datetime, timedelta
import json
import re
import warnings
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import timm
from PIL import Image
import io
import base64
from typing import List, Dict, Tuple, Optional, Any, Union
import hashlib
import sqlite3
import os
from pathlib import Path
from functools import lru_cache
import uuid
import whisper
import soundfile as sf
import librosa
import time
import asyncio
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import optuna
import gc
import sys

warnings.filterwarnings('ignore')

# Import required libraries for RAG
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import faiss

# Setup environment
os.environ['TRANSFORMERS_CACHE'] = '/content/cache'
os.environ['HF_HOME'] = '/content/cache'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Enable optimizations
if torch.cuda.is_available():
    print(f"✅ CUDA available - GPU: {torch.cuda.get_device_name()}")
    torch.backends.cuda.enable_flash_sdp(True)
    torch.backends.cuda.enable_mem_efficient_sdp(True)
    torch.backends.cuda.enable_math_sdp(True)
else:
    print("⚠️ CUDA not available - using CPU")

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")

# Thai NLP Setup with fallback
try:
    from pythainlp.tokenize import word_tokenize
    from pythainlp.corpus.common import thai_stopwords
    from pythainlp.spell import correct
    from pythainlp.util import normalize
    print("✅ PyThaiNLP imported successfully")
    THAI_NLP_AVAILABLE = True
except Exception as e:
    print(f"⚠️ PyThaiNLP setup issue: {e}")
    THAI_NLP_AVAILABLE = False

    def word_tokenize(text, engine='basic'):
        # Simple Thai tokenization fallback
        import re
        tokens = re.findall(r'[ก-๙]+|[a-zA-Z]+|\d+', text)
        return tokens if tokens else text.split()

    def thai_stopwords():
        return {'และ', 'หรือ', 'ที่', 'ใน', 'จาก', 'เป็น', 'มี', 'ได้', 'ไป', 'มา'}

    def correct(text):
        return text

    def normalize(text):
        return text

In [ ]:
# Mount Google Drive with error handling
def setup_drive_connection():
    """Setup Google Drive connection with fallback"""
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        drive_path = "/content/drive/MyDrive/V89Technology/typhoon21-gemma3-4b-medCare-finetuned/checkpoint-120"
        print("Google Drive connected successfully")
        return drive_path
    except:
        local_path = "V89Technology/typhoon21-gemma3-4b-medCare-finetuned/checkpoint-120"
        os.makedirs(local_path, exist_ok=True)
        print("Running outside Colab - using local directory")
        return local_path

drive_path = setup_drive_connection()

# Configuration
GOOGLE_DRIVE_FOLDER = "V89Technology"
RAG_THRESHOLD = 0.7
MAX_TOKENS = 512
TOP_K = 5
PDPA_CONSENT_VERSION = "1.0"
MEDICAL_STANDARD_VERSION = "HL7_FHIR_R4"
CACHE_SIZE = 1000
BATCH_SIZE = 32

class GoogleDriveManager:
    """Simplified Google Drive Manager for demo purposes"""

    def __init__(self, folder_name=GOOGLE_DRIVE_FOLDER):
        self.folder_name = folder_name
        self.setup_local_storage()

    def setup_local_storage(self):
        """Setup local storage for demo"""
        try:
            os.makedirs(self.folder_name, exist_ok=True)
            print(f"✅ Local storage created: {self.folder_name}")
        except Exception as e:
            print(f"⚠️ Storage setup failed: {e}")

    def save_patient_data(self, patient_id: str, data: dict):
        """Save patient data locally with PDPA compliance"""
        try:
            filename = os.path.join(self.folder_name, f"patient_{patient_id}.json")
            encrypted_data = self.encrypt_patient_data(data)
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(encrypted_data, f, ensure_ascii=False, indent=2)
            print(f"✅ Patient data saved: {patient_id}")
            return True
        except Exception as e:
            print(f"❌ Error saving patient data: {e}")
            return False

    def load_patient_data(self, patient_id: str) -> dict:
        """Load patient data locally"""
        try:
            filename = os.path.join(self.folder_name, f"patient_{patient_id}.json")
            if os.path.exists(filename):
                with open(filename, 'r', encoding='utf-8') as f:
                    encrypted_data = json.load(f)
                return self.decrypt_patient_data(encrypted_data)
        except Exception as e:
            print(f"❌ Error loading patient data: {e}")
        return {}

    def encrypt_patient_data(self, data: dict) -> dict:
        """Encrypt sensitive patient data for PDPA compliance"""
        try:
            return {
                'data_hash': hashlib.sha256(json.dumps(data, ensure_ascii=False).encode()).hexdigest(),
                'data': data,
                'encrypted_at': datetime.now().isoformat(),
                'pdpa_version': PDPA_CONSENT_VERSION
            }
        except Exception as e:
            print(f"❌ Error encrypting data: {e}")
            return data

    def decrypt_patient_data(self, encrypted_data: dict) -> dict:
        """Decrypt patient data"""
        try:
            if 'data' not in encrypted_data:
                return encrypted_data

            # Verify data integrity
            if 'data_hash' in encrypted_data:
                current_hash = hashlib.sha256(json.dumps(encrypted_data['data'], ensure_ascii=False).encode()).hexdigest()
                if current_hash != encrypted_data['data_hash']:
                    print("⚠️ Data integrity check failed")

            return encrypted_data['data']
        except Exception as e:
            print(f"❌ Error decrypting data: {e}")
            return {}

class EmergencyTriageSystem:
    """Real-time Emergency Triage System"""

    def __init__(self):
        self.triage_criteria = {
            'critical': ['เจ็บหน้าอก', 'หายใจไม่ออก', 'หมดสติ', 'ชัก', 'เลือดออกมาก', 'แขนขาอ่อนแรง', 'เจ็บหน้าอกรุนแรง'],
            'high': ['ไข้สูง', 'ปวดหัวรุนแรง', 'อาเจียนเป็นเลือด', 'ปวดท้องรุนแรง', 'ตาเหลือง', 'หอบเหนื่อย'],
            'medium': ['ปวดท้อง', 'ไข้', 'ผื่น', 'ไอ', 'น้ำมูก', 'ท้องเสีย'],
            'low': ['อาการเล็กน้อย', 'คำถามทั่วไป', 'ปรึกษาทั่วไป', 'ตรวจสุขภาพ']
        }
        self.emergency_response = {
            'critical': '⚠️ อาการฉุกเฉิน! กรุณาโทร 1669 หรือไปโรงพยาบาลทันที',
            'high': '⚠️ อาการรุนแรง ควรพบแพทย์ภายใน 24 ชั่วโมง',
            'medium': '📋 อาการปานกลาง ควรพบแพทย์หากอาการไม่ดีขึ้นใน 2-3 วัน',
            'low': '💡 อาการเบื้องต้น สามารถสังเกตอาการที่บ้านได้'
        }

    def assess_urgency(self, text, image_analysis=None, audio_analysis=None):
        """Assess urgency level based on input"""
        text_lower = text.lower()

        # Check for critical keywords
        for keyword in self.triage_criteria['critical']:
            if keyword in text_lower:
                return 'critical'

        # Check for high priority keywords
        high_score = sum(1 for keyword in self.triage_criteria['high'] if keyword in text_lower)
        if high_score >= 2:
            return 'high'

        # Check for medium priority keywords
        medium_score = sum(1 for keyword in self.triage_criteria['medium'] if keyword in text_lower)
        if medium_score >= 1:
            return 'medium'

        return 'low'

    def get_emergency_response(self, urgency_level):
        """Get appropriate emergency response"""
        return self.emergency_response.get(urgency_level, '💡 อาการเบื้องต้น สามารถสังเกตอาการที่บ้านได้')

class AdvancedSymptomChecker:
    """Advanced Symptom Checker with Thai medical knowledge"""

    def __init__(self):
        self.symptom_ontology = {
            'fever': ['ไข้', 'ตัวร้อน', 'ร้อนใน', 'ไข้สูง'],
            'headache': ['ปวดหัว', 'ศีรษะ', 'มึนหัว', 'หนาวหัว'],
            'cough': ['ไอ', 'เสมหะ', 'ไอแห้ง', 'ไอมีเสมหะ'],
            'abdominal_pain': ['ปวดท้อง', 'ท้องเสีย', 'ถ่ายเหลว', 'ปวดกระเพาะ'],
            'chest_pain': ['เจ็บหน้าอก', 'ปวดหน้าอก', 'หน้าอกตึง'],
            'shortness_of_breath': ['หอบเหนื่อย', 'หายใจลำบาก', 'หายใจไม่ออก'],
            'rash': ['ผื่น', 'คัน', 'ลมพิษ', 'แผลผื่น']
        }

        self.disease_symptom_map = {
            'influenza': ['fever', 'headache', 'cough'],
            'dengue': ['fever', 'headache', 'rash'],
            'gastroenteritis': ['abdominal_pain', 'fever'],
            'heart_attack': ['chest_pain', 'shortness_of_breath'],
            'allergy': ['rash', 'cough']
        }

    def analyze_symptoms(self, symptoms: List[str], patient_info: Dict) -> Dict:
        """Analyze symptoms and provide assessment"""
        matched_conditions = self._match_symptoms_to_conditions(symptoms)
        risk_score = self._calculate_risk_score(symptoms, patient_info)

        return {
            'possible_conditions': matched_conditions,
            'risk_score': risk_score,
            'recommendations': self._generate_recommendations(matched_conditions, risk_score)
        }

    def _match_symptoms_to_conditions(self, symptoms: List[str]) -> List[str]:
        """Match symptoms to possible medical conditions"""
        matched = []
        symptom_keys = []

        # Convert symptoms to ontology keys
        for symptom in symptoms:
            for key, thai_terms in self.symptom_ontology.items():
                if any(term in symptom for term in thai_terms):
                    symptom_keys.append(key)

        # Find matching conditions
        for condition, condition_symptoms in self.disease_symptom_map.items():
            match_count = sum(1 for symptom in symptom_keys if symptom in condition_symptoms)
            if match_count >= 1:
                matched.append(condition)

        return matched

    def _calculate_risk_score(self, symptoms: List[str], patient_info: Dict) -> float:
        """Calculate risk score"""
        base_score = len(symptoms) * 0.1
        age = patient_info.get('age', 30)

        if age < 5 or age > 65:
            base_score += 0.2

        if patient_info.get('medical_history'):
            base_score += 0.2

        return min(base_score, 1.0)

    def _generate_recommendations(self, conditions: List[str], risk_score: float) -> List[str]:
        """Generate medical recommendations"""
        recommendations = []

        if risk_score > 0.7:
            recommendations.append("ควรพบแพทย์โดยเร็ว")
        elif risk_score > 0.4:
            recommendations.append("ควรติดตามอาการและพบแพทย์หากอาการแย่ลง")
        else:
            recommendations.append("สังเกตอาการและดูแลตัวเอง")

        if 'heart_attack' in conditions:
            recommendations.append("ฉุกเฉิน! โทร 1669 ทันที")

        return recommendations

class EnhancedMultiModalProcessor:
    """Enhanced MultiModal Processor for medical analysis"""

    def __init__(self):
        self.setup_models()

    def setup_models(self):
        """Setup vision and audio models"""
        # Vision model
        try:
            self.vision_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
            self.vision_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
            print("✅ Vision model loaded")
        except Exception as e:
            print(f"⚠️ Vision model failed: {e}")
            self.vision_model = None

        # Whisper model
        try:
            self.whisper_model = whisper.load_model("base")
            print("✅ Whisper audio model loaded")
        except Exception as e:
            print(f"⚠️ Whisper model failed: {e}")
            self.whisper_model = None

    def process_image(self, image_path: str) -> Dict:
        """Process medical image"""
        if not self.vision_model or not image_path:
            return {'error': 'Vision model not available or no image provided'}

        try:
            image = Image.open(image_path).convert('RGB')
            inputs = self.vision_processor(images=image, return_tensors="pt")

            with torch.no_grad():
                outputs = self.vision_model(**inputs)

            logits = outputs.logits
            predicted_class_idx = logits.argmax(-1).item()
            confidence = torch.nn.functional.softmax(logits, dim=-1).max().item()

            return {
                'status': 'success',
                'prediction': self.vision_model.config.id2label[predicted_class_idx],
                'confidence': float(confidence),
                'analysis': 'ผลการวิเคราะห์ภาพเบื้องต้น - ควรปรึกษาแพทย์สำหรับการวินิจฉัยที่แม่นยำ'
            }

        except Exception as e:
            return {'error': f'Image processing failed: {str(e)}'}

    def process_audio(self, audio_path: str) -> Dict:
        """Process audio to text using Whisper"""
        if not self.whisper_model or not audio_path:
            return {'error': 'Whisper model not available or no audio provided'}

        try:
            result = self.whisper_model.transcribe(audio_path)

            return {
                'status': 'success',
                'text': result['text'],
                'language': result.get('language', 'unknown'),
                'confidence': 0.9  # Whisper doesn't provide confidence directly
            }
        except Exception as e:
            return {'error': f'Audio processing failed: {str(e)}'}

class EnhancedMedicalRAGSystem:
    """Enhanced RAG System with Medical Knowledge Base"""

    def __init__(self, drive_manager: GoogleDriveManager):
        self.drive_manager = drive_manager
        self.embedding_model = None
        self.medical_documents = []
        self.chroma_client = None
        self.collection = None
        self.triage_system = EmergencyTriageSystem()
        self.symptom_checker = AdvancedSymptomChecker()
        self.setup_system()

    def setup_system(self):
        """Setup the complete RAG system"""
        self.setup_embedding_model()
        self.setup_vector_database()
        self.load_medical_knowledge()
        self.process_documents_for_rag()

    def setup_embedding_model(self):
        """Setup multilingual embedding model"""
        try:
            model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
            self.embedding_model = SentenceTransformer(model_name)
            print("✅ Embedding model loaded")
        except Exception as e:
            print(f"❌ Error loading embedding model: {e}")

    def setup_vector_database(self):
        """Setup ChromaDB vector database"""
        try:
            self.chroma_client = chromadb.PersistentClient(path="./chroma_db")

            try:
                self.collection = self.chroma_client.get_collection(name="medical_knowledge")
                print("✅ Existing collection loaded")
            except:
                self.collection = self.chroma_client.create_collection(
                    name="medical_knowledge",
                    metadata={"hnsw:space": "cosine"}
                )
                print("✅ New collection created")

        except Exception as e:
            print(f"❌ Error setting up vector database: {e}")

    def load_medical_knowledge(self):
        """Load comprehensive medical knowledge base"""
        print("📚 Loading medical knowledge base...")

        # Thai medical standards and guidelines
        thai_medical_data = [
            {
                'content': 'โปรโตคอลฉุกเฉิน: อาการเจ็บหน้าอกรุนแรง - ให้ออกซิเจน, ตรวจ EKG ทันที, ให้ Aspirin 300mg เคี้ยว, เตรียมส่งต่อ PCI',
                'category': 'emergency',
                'type': 'protocol'
            },
            {
                'content': 'การประเมิน ABCDE: Airway (ทางเดินหายใจ), Breathing (การหายใจ), Circulation (ระบบไหลเวียน), Disability (ระบบประสาท), Exposure (ตรวจร่างกาย)',
                'category': 'emergency',
                'type': 'assessment'
            },
            {
                'content': 'แนวทางการรักษาเบาหวาน: เป้าหมาย HbA1c < 7%, FBS 80-130 mg/dL, 2hr PP < 180 mg/dL, ตรวจตา/ไต/เท้าปีละครั้ง',
                'category': 'endocrine',
                'type': 'guideline'
            },
            {
                'content': 'แนวทางความดันโลหิตสูง: เป้าหมาย BP < 140/90 mmHg (ทั่วไป), < 130/80 mmHg (เบาหวาน/โรคไต), ปรับวิถีชีวิต + ยา',
                'category': 'cardiovascular',
                'type': 'guideline'
            }
        ]

        # Medical Q&A data
        medical_qa = [
            {
                'content': 'Q: อาการของไข้หวัดใหญ่คืออะไร A: อาการไข้หวัดใหญ่ ได้แก่ ไข้สูง หนาวสั่น ปวดกล้ามเนื้อ ปวดหัว เหนื่อยอ่อน ไอแห้ง น้ำมูกไหล คออักเสบ อาการจะรุนแรงกว่าหวัดธรรมดา',
                'category': 'infectious_disease',
                'type': 'qa'
            },
            {
                'content': 'Q: วิธีดูแลผู้ป่วยเบาหวาน A: การดูแลผู้ป่วยเบาหวาน: ควบคุมน้ำตาลในเลือด รับประทานยาสม่ำเสมอ ออกกำลังกายสม่ำเสมอ รับประทานอาหารตามหลักโภชนาการ ตรวจสุขภาพเป็นระยะ',
                'category': 'endocrine',
                'type': 'qa'
            }
        ]

        # Drug interaction data
        drug_interactions = [
            {
                'content': 'Warfarin + Aspirin: เพิ่มความเสี่ยงเลือดออก ควรหลีกเลี่ยงการใช้ร่วมกัน หรือปรับขนาดยาและติดตามค่า INR อย่างใกล้ชิด',
                'category': 'drug_interaction',
                'type': 'safety'
            },
            {
                'content': 'Metformin + Contrast dye: เพิ่มความเสี่ยง lactic acidosis ควรหยุด metformin 48 ชั่วโมงก่อนและหลังให้ contrast',
                'category': 'drug_interaction',
                'type': 'safety'
            }
        ]

        # Symptom-disease mapping
        symptom_mapping = [
            {
                'content': 'กลุ่มอาการไข้สูง ปวดหัว ปวดกล้ามเนื้อ น่าสงสัยไข้หวัดใหญ่ ควรตรวจ Rapid influenza test และรักษาตามอาการ',
                'category': 'diagnostic',
                'type': 'symptom_mapping'
            },
            {
                'content': 'อาการปวดท้องร่วมกับคลื่นไส้อาเจียน น่าสงสัยกระเพาะอาหารอักเสบหรือกรดไหลย้อน ควรปรับอาหารและให้ยาลดกรด',
                'category': 'diagnostic',
                'type': 'symptom_mapping'
            }
        ]

        # Combine all medical data
        all_medical_data = thai_medical_data + medical_qa + drug_interactions + symptom_mapping

        # Convert to document format
        for item in all_medical_data:
            self.medical_documents.append({
                'content': item['content'],
                'metadata': {
                    'source': 'medical_knowledge_base',
                    'category': item['category'],
                    'type': item['type'],
                    'language': 'thai',
                    'updated': datetime.now().isoformat()
                }
            })

        print(f"✅ Loaded {len(self.medical_documents)} medical documents")

    def process_documents_for_rag(self):
        """Process and index documents for RAG"""
        if not self.embedding_model or not self.collection:
            print("❌ Cannot process documents - missing components")
            return

        try:
            print(f"📊 Processing {len(self.medical_documents)} documents for RAG...")

            # Clear existing collection
            try:
                self.collection.delete(where={})
            except:
                pass

            # Process in batches to avoid memory issues
            batch_size = 50
            for i in range(0, len(self.medical_documents), batch_size):
                batch = self.medical_documents[i:i+batch_size]

                contents = [doc['content'] for doc in batch]
                embeddings = self.embedding_model.encode(contents).tolist()

                # Add to ChromaDB
                self.collection.add(
                    documents=contents,
                    embeddings=embeddings,
                    metadatas=[doc['metadata'] for doc in batch],
                    ids=[str(uuid.uuid4()) for _ in batch]
                )

            print(f"✅ Successfully indexed {len(self.medical_documents)} documents")

        except Exception as e:
            print(f"❌ Error processing documents: {e}")

    def retrieve_relevant_documents(self, query: str, top_k: int = 5) -> List[Dict]:
        """Retrieve relevant medical documents"""
        if not self.embedding_model or not self.collection:
            return []

        try:
            query_embedding = self.embedding_model.encode([query]).tolist()[0]

            results = self.collection.query(
                query_embeddings=[query_embedding],
                n_results=top_k
            )

            relevant_docs = []
            if results and results['documents']:
                for i, doc in enumerate(results['documents'][0]):
                    relevant_docs.append({
                        'content': doc,
                        'metadata': results['metadatas'][0][i] if results['metadatas'] else {},
                        'score': 1 - results['distances'][0][i] if results['distances'] else 0.5
                    })

            return relevant_docs

        except Exception as e:
            print(f"❌ Error retrieving documents: {e}")
            return []

    def generate_medical_response(self, query: str, patient_info: Dict = None) -> Dict:
        """Generate comprehensive medical response"""
        if not patient_info:
            patient_info = {}

        # Retrieve relevant knowledge
        relevant_docs = self.retrieve_relevant_documents(query, top_k=TOP_K)

        # Assess urgency
        urgency_level = self.triage_system.assess_urgency(query)
        emergency_response = self.triage_system.get_emergency_response(urgency_level)

        # Analyze symptoms
        symptoms = self._extract_symptoms(query)
        symptom_analysis = self.symptom_checker.analyze_symptoms(symptoms, patient_info)

        # Generate response
        response = self._format_comprehensive_response(
            query=query,
            relevant_docs=relevant_docs,
            urgency_level=urgency_level,
            emergency_response=emergency_response,
            symptom_analysis=symptom_analysis
        )

        return response

    def _extract_symptoms(self, text: str) -> List[str]:
        """Extract symptoms from Thai text"""
        symptoms = []
        symptom_keywords = [
            'ปวด', 'ไข้', 'ร้อน', 'หนาว', 'ไอ', 'จาม', 'น้ำมูก',
            'คัดจมูก', 'เจ็บ', 'คัน', 'ผื่น', 'แดง', 'บวม',
            'คลื่นไส้', 'อาเจียน', 'ท้องเสีย', 'ท้องผูก',
            'เวียนหัว', 'มึนหัว', 'เหนื่อย', 'อ่อนเพลีย',
            'หอบเหนื่อย', 'หายใจลำบาก'
        ]

        for keyword in symptom_keywords:
            if keyword in text:
                # Extract context around symptom
                start = max(0, text.find(keyword) - 15)
                end = min(len(text), text.find(keyword) + len(keyword) + 15)
                symptom_phrase = text[start:end].strip()
                symptoms.append(symptom_phrase)

        return symptoms

    def _format_comprehensive_response(self, **kwargs) -> Dict:
        """Format comprehensive medical response"""
        response_parts = []

        # Emergency alert
        if kwargs['urgency_level'] in ['critical', 'high']:
            response_parts.append(f"🚨 {kwargs['emergency_response']}")

        # Relevant medical information
        if kwargs['relevant_docs']:
            response_parts.append("📚 **ข้อมูลทางการแพทย์ที่เกี่ยวข้อง:**")
            for doc in kwargs['relevant_docs'][:3]:  # Top 3 relevant docs
                if doc['score'] > RAG_THRESHOLD:
                    response_parts.append(f"- {doc['content'][:200]}...")

        # Symptom analysis
        if kwargs['symptom_analysis']['possible_conditions']:
            response_parts.append("🔍 **การวิเคราะห์อาการ:**")
            conditions = ', '.join(kwargs['symptom_analysis']['possible_conditions'])
            response_parts.append(f"อาการที่พบอาจเกี่ยวข้องกับ: {conditions}")

        # Recommendations
        if kwargs['symptom_analysis']['recommendations']:
            response_parts.append("💡 **คำแนะนำ:**")
            for rec in kwargs['symptom_analysis']['recommendations']:
                response_parts.append(f"- {rec}")

        # Medical disclaimer
        response_parts.append("\n⚠️ **ข้อปฏิเสธความรับผิดชอบ:** ข้อมูลนี้เป็นเพียงคำแนะนำเบื้องต้น ไม่ใช่การวินิจฉัยทางการแพทย์ กรุณาปรึกษาแพทย์สำหรับการรักษาที่เหมาะสม")

        return {
            'response': '\n\n'.join(response_parts),
            'urgency_level': kwargs['urgency_level'],
            'confidence': kwargs['symptom_analysis']['risk_score'],
            'relevant_sources': len(kwargs['relevant_docs']),
            'timestamp': datetime.now().isoformat()
        }

class ThaiMedicalAIAgent:
    """Main Thai Medical AI Agent with complete functionality"""

    def __init__(self):
        print("🏥 Initializing Thai Medical AI Agent...")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"💻 Using device: {self.device}")

        # Initialize components
        self.drive_manager = GoogleDriveManager()
        self.rag_system = EnhancedMedicalRAGSystem(self.drive_manager)
        self.multimodal_processor = EnhancedMultiModalProcessor()

        # Initialize language model
        self.setup_language_model()

        # Patient session management
        self.current_patient = {}
        self.conversation_history = []

        print("✅ Thai Medical AI Agent initialized successfully!")

    def setup_language_model(self):
        """Load typhoon21-gemma3-4b model with PEFT adapter medical fine-tuning"""
        try:
            base_model_name = "scb10x/typhoon2.1-gemma3-4b"
            adapter_path = "/content/drive/MyDrive/V89Technology/typhoon21-gemma3-4b-medCare-finetuned/checkpoint-120"

            # Configure for T4 GPU efficiency
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4"
            )

            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Load base model
            self.model = AutoModelForCausalLM.from_pretrained(
                base_model_name,
               quantization_config=quantization_config,
                device_map="auto",
                torch_dtype=torch.bfloat16,
                trust_remote_code=True,
                low_cpu_mem_usage=True
            )

            # Load PEFT adapter
            from peft import PeftModel
            self.model = PeftModel.from_pretrained(
                self.model,
                adapter_path,
                adapter_name="medical_adapter"
            )

            # Set model to evaluation mode
            self.model.eval()

            print("✅ typhoon21-gemma3-4b model with medical PEFT adapter loaded successfully")

        except Exception as e:
            print(f"⚠️ Error loading typhoon21-gemma3-4b LLM: {e}")
            # Fallback to pipeline
            try:
                self.pipeline = pipeline(
                    "text-generation",
                    model="scb10x/typhoon2.1-gemma3-4b",
                    device=0 if torch.cuda.is_available() else -1
                )
                self.model = None
                self.tokenizer = None
                print("✅ Fallback model loaded")
            except:
                self.model = None
                self.tokenizer = None
                self.pipeline = None
                print("❌ No language model available")

    def process_patient_info(self, name: str, age: int, gender: str, medical_history: str,
                           current_symptoms: str, consent_pdpa: bool) -> str:
        """Process and store patient information with PDPA compliance"""
        if not consent_pdpa:
            return "❌ จำเป็นต้องยินยอมการใช้ข้อมูลส่วนบุคคลตาม พ.ร.บ. PDPA เพื่อใช้บริการ"

        try:
            # Generate patient ID
            patient_id = str(uuid.uuid4())[:8]

            # Store patient information
            patient_data = {
                'patient_id': patient_id,
                'name': name,
                'age': age,
                'gender': gender,
                'medical_history': medical_history,
                'current_symptoms': current_symptoms,
                'created_at': datetime.now().isoformat(),
                'pdpa_consent': consent_pdpa,
                'pdpa_version': PDPA_CONSENT_VERSION
            }

            # Save to secure storage
            success = self.drive_manager.save_patient_data(patient_id, patient_data)

            if success:
                self.current_patient = patient_data

                # Initial triage assessment
                urgency = self.rag_system.triage_system.assess_urgency(current_symptoms)

                response = f"""✅ **ข้อมูลผู้ป่วยถูกบันทึกแล้ว**

👤 **ข้อมูลทั่วไป:**
- ID: {patient_id}
- ชื่อ: {name}
- อายุ: {age} ปี
- เพศ: {gender}

🏥 **การประเมินเบื้องต้น:**
- ระดับความเร่งด่วน: {urgency}
- {self.rag_system.triage_system.get_emergency_response(urgency)}

💬 **พร้อมให้คำปรึกษา:** คุณสามารถสอบถามเกี่ยวกับอาการหรือปัญหาสุขภาพได้เลย
"""

                return response
            else:
                return "❌ เกิดข้อผิดพลาดในการบันทึกข้อมูล กรุณาลองใหม่อีกครั้ง"

        except Exception as e:
            print(f"❌ Error processing patient info: {e}")
            return f"❌ เกิดข้อผิดพลาด: {str(e)}"

    def process_chat(self, user_input: str, chat_history: List) -> Tuple[str, List]:
        """Process chat with medical context and RAG"""
        try:
            if not user_input.strip():
                return "กรุณาพิมพ์คำถามหรืออาการที่ต้องการสอบถาม", chat_history

            # Generate medical response using RAG
            medical_response = self.rag_system.generate_medical_response(
                query=user_input,
                patient_info=self.current_patient
            )

            # Enhanced response with LLM if available
            if self.model and self.tokenizer:
                enhanced_response = self._enhance_response_with_llm(
                    user_input, medical_response['response']
                )
            else:
                enhanced_response = medical_response['response']

            # Update conversation history
            chat_history.append((user_input, enhanced_response))
            self.conversation_history.append({
                'user': user_input,
                'assistant': enhanced_response,
                'timestamp': datetime.now().isoformat(),
                'urgency_level': medical_response['urgency_level']
            })

            return enhanced_response, chat_history

        except Exception as e:
            error_msg = f"เกิดข้อผิดพลาด: {str(e)}"
            chat_history.append((user_input, error_msg))
            return error_msg, chat_history

    def _enhance_response_with_llm(self, user_input: str, rag_response: str) -> str:
        """Enhance RAG response with language model"""
        try:
            # Create medical context prompt
            system_prompt = """คุณเป็นผู้ช่วยแพทย์ AI ที่ช่วยให้คำปรึกษาทางการแพทย์เบื้องต้นในภาษาไทย
โปรดตอบคำถามอย่างระมัดระวัง โดยอิงจากข้อมูลทางการแพทย์ที่ถูกต้อง และเตือนให้ผู้ป่วยปรึกษาแพทย์เมื่อจำเป็น"""

            # Combine context
            context = f"{system_prompt}\n\nข้อมูลอ้างอิง: {rag_response}\n\nคำถาม: {user_input}\n\nคำตอบ:"

            # Tokenize
            inputs = self.tokenizer.encode(context, return_tensors="pt", max_length=512, truncation=True)

            if torch.cuda.is_available():
                inputs = inputs.to(self.device)

            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 150,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    no_repeat_ngram_size=3
                )

            # Decode response
            generated = self.tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

            # Clean and format response
            if generated.strip():
                return f"{rag_response}\n\n💭 **เพิ่มเติม:** {generated.strip()}"
            else:
                return rag_response

        except Exception as e:
            print(f"⚠️ LLM enhancement failed: {e}")
            return rag_response

    def process_image(self, image_file) -> str:
        """Process medical image"""
        if image_file is None:
            return "❌ กรุณาอัปโหลดไฟล์ภาพ"

        try:
            # Save uploaded image temporarily
            temp_path = f"/tmp/uploaded_image_{int(time.time())}.jpg"

            # Handle different input types
            if hasattr(image_file, 'name'):
                # Gradio file object
                image = Image.open(image_file.name)
            else:
                # Direct path or PIL image
                image = Image.open(image_file) if isinstance(image_file, str) else image_file

            # Convert to RGB and save
            image = image.convert('RGB')
            image.save(temp_path)

            # Process with multimodal processor
            result = self.multimodal_processor.process_image(temp_path)

            # Clean up
            try:
                os.remove(temp_path)
            except:
                pass

            if 'error' in result:
                return f"❌ เกิดข้อผิดพลาด: {result['error']}"
            else:
                return f"""📊 **ผลการวิเคราะห์ภาพ:**

🔍 **การวินิจฉMเบื้องต้น:** {result['prediction']}
📈 **ความมั่นใจ:** {result['confidence']:.2%}
💡 **คำแนะนำ:** {result['analysis']}

⚠️ **หมายเหตุ:** นี่เป็นการวิเคราะห์เบื้องต้นเท่านั้น กรุณาปรึกษาแพทย์สำหรับการวินิจฉัยที่แม่นยำ
"""

        except Exception as e:
            return f"❌ เกิดข้อผิดพลาดในการประมวลผลภาพ: {str(e)}"

    def process_audio(self, audio_file) -> str:
        """Process audio input"""
        if audio_file is None:
            return "❌ กรุณาอัปโหลดไฟล์เสียง"

        try:
            # Process audio with Whisper
            result = self.multimodal_processor.process_audio(audio_file)

            if 'error' in result:
                return f"❌ เกิดข้อผิดพลาด: {result['error']}"
            else:
                # Continue with chat processing using transcribed text
                transcribed_text = result['text']
                response, _ = self.process_chat(transcribed_text, [])

                return f"""🎤 **ข้อความจากเสียง:** {transcribed_text}

{response}
"""

        except Exception as e:
            return f"❌ เกิดข้อผิดพลาดในการประมวลผลเสียง: {str(e)}"

    def get_patient_summary(self) -> str:
        """Get current patient summary"""
        if not self.current_patient:
            return "❌ ไม่มีข้อมูลผู้ป่วยในระบบ"

        patient = self.current_patient
        return f"""📋 **สรุปข้อมูลผู้ป่วย:**

👤 **ข้อมูลส่วนตัว:**
- ID: {patient.get('patient_id', 'N/A')}
- ชื่อ: {patient.get('name', 'N/A')}
- อายุ: {patient.get('age', 'N/A')} ปี
- เพศ: {patient.get('gender', 'N/A')}

🏥 **ประวัติสุขภาพ:**
{patient.get('medical_history', 'ไม่มีข้อมูล')}

🤒 **อาการปัจจุบัน:**
{patient.get('current_symptoms', 'ไม่มีข้อมูล')}

📅 **สร้างเมื่อ:** {patient.get('created_at', 'N/A')}
"""

    def clear_conversation(self) -> str:
        """Clear conversation history"""
        self.conversation_history = []
        return "🗑️ ล้างประวัติการสนทนาเรียบร้อยแล้ว"

    def emergency_triage(self, symptoms: str) -> str:
        """Emergency triage assessment"""
        urgency = self.rag_system.triage_system.assess_urgency(symptoms)
        response = self.rag_system.triage_system.get_emergency_response(urgency)

        return f"""🚨 **การประเมินฉุกเฉิน:**

🔍 **อาการที่รายงาน:** {symptoms}
📊 **ระดับความเร่งด่วน:** {urgency}
💡 **คำแนะนำ:** {response}

{'🚑 **กรุณาโทร 1669 หรือไปโรงพยาบาลทันที**' if urgency == 'critical' else 'โปรดติดตามอาการอย่างใกล้ชิด'}
"""

# Initialize the AI Agent
ai_agent = ThaiMedicalAIAgent()

# Create Gradio interface
def create_gradio_interface():
    """Create comprehensive Gradio interface"""

    with gr.Blocks(
        title="Thai Medical Care AI Agent - V89 Technology",
        theme=gr.themes.Soft(),
        css="""
        .medical-chat { background-color: #f0f8ff; }
        .emergency { background-color: #ffcccc; padding: 10px; border-radius: 5px; }
        .success { background-color: #ccffcc; padding: 10px; border-radius: 5px; }
        .warning { background-color: #ffffcc; padding: 10px; border-radius: 5px; }
        """
    ) as demo:

        gr.Markdown("""
        # 🏥 Thai Medical Care AI Agent Chat with RAG System
        ### **บริษัท วี89 เทคโนโลยี จำกัด** | **📈เวอร์ชัน:** 2.0 - อัปเดตล่าสุด: สิงหาคม 2025

        🤖  ระบบผู้ช่วยทางการแพทย์อัจฉริยะสำหรับวิเคราะห์อาการเบื้องต้นและให้คำแนะนำทางการแพทย์ภาษาไทย พร้อม RAG
        """)

        with gr.Tab("📋 ลงทะเบียนผู้ป่วย"):
            gr.Markdown("### 📝 ลงทะเบียนข้อมูลผู้ป่วย")

            with gr.Row():
                with gr.Column():
                    name_input = gr.Textbox(label="ชื่อ-นามสกุล", placeholder="กรุณากรอกชื่อและนามสกุล")
                    age_input = gr.Number(label="อายุ", minimum=0, maximum=120)
                    gender_input = gr.Radio(choices=["ชาย", "หญิง", "อื่นๆ"], label="เพศ")

                with gr.Column():
                    medical_history_input = gr.Textbox(
                        label="ประวัติการเจ็บป่วย",
                        placeholder="โรคประจำตัว, การแพ้ยา, ประวัติการผ่าตัด ฯลฯ",
                        lines=3
                    )
                    current_symptoms_input = gr.Textbox(
                        label="อาการปัจจุบัน",
                        placeholder="อธิบายอาการที่กำลังประสบอยู่",
                        lines=3
                    )
                    consent_checkbox = gr.Checkbox(
                        label="ฉันยินยอมให้เก็บและใช้ข้อมูลตาม พ.ร.บ. คุ้มครองข้อมูลส่วนบุคคล พ.ศ. 2562",
                        value=True
                    )

            register_btn = gr.Button("✅ ลงทะเบียนผู้ป่วย", variant="primary")
            patient_output = gr.Markdown()

            register_btn.click(
                ai_agent.process_patient_info,
                inputs=[name_input, age_input, gender_input, medical_history_input, current_symptoms_input, consent_checkbox],
                outputs=patient_output
            )

        with gr.Tab("💬 ปรึกษาแพทย์ AI"):
            gr.Markdown("### 💬 สนทนากับผู้ช่วยแพทย์ AI")

            chatbot = gr.Chatbot(
                label="การสนทนา",
                height=400,
                elem_classes="medical-chat"
            )

            with gr.Row():
                msg = gr.Textbox(
                    label="พิมพ์คำถามหรืออาการของคุณ",
                    placeholder="เช่น 'วันนี้ฉันปวดหัวและมีไข้ ควรทำอย่างไรดี?'",
                    scale=4
                )
                send_btn = gr.Button("📤 ส่ง", scale=1)

            with gr.Row():
                clear_btn = gr.Button("🗑️ ล้างการสนทนา")
                summary_btn = gr.Button("📋 แสดงข้อมูลผู้ป่วย")

            patient_summary = gr.Markdown()

            def respond(message, chat_history):
                response, updated_chat = ai_agent.process_chat(message, chat_history)
                return updated_chat

            msg.submit(respond, [msg, chatbot], [chatbot])
            send_btn.click(respond, [msg, chatbot], [chatbot])
            clear_btn.click(lambda: None, None, [chatbot], queue=False)
            clear_btn.click(ai_agent.clear_conversation, None, [patient_summary])
            summary_btn.click(ai_agent.get_patient_summary, None, [patient_summary])

        with gr.Tab("🖼️ วิเคราะห์ภาพ"):
            gr.Markdown("### 🖼️ วิเคราะห์ภาพทางการแพทย์เบื้องต้น")

            with gr.Row():
                image_input = gr.Image(
                    label="อัปโหลดภาพ",
                    type="filepath",
                    sources=["upload", "webcam"]
                )
                image_output = gr.Markdown()

            analyze_btn = gr.Button("🔍 วิเคราะห์ภาพ", variant="primary")
            analyze_btn.click(ai_agent.process_image, inputs=[image_input], outputs=[image_output])

        with gr.Tab("🎤 ปรึกษาด้วยเสียง"):
            gr.Markdown("### 🎤 สนทนาด้วยเสียง")

            with gr.Row():
                audio_input = gr.Audio(
                    label="อัปโหลดไฟล์เสียงหรือบันทึกเสียง",
                    sources=["upload", "microphone"],
                    type="filepath"
                )
                audio_output = gr.Markdown()

            process_audio_btn = gr.Button("🔊 ประมวลผลเสียง", variant="primary")
            process_audio_btn.click(ai_agent.process_audio, inputs=[audio_input], outputs=[audio_output])

        with gr.Tab("🚨 ประเมินฉุกเฉิน"):
            gr.Markdown("### 🚨 การประเมินอาการฉุกเฉิน")

            emergency_input = gr.Textbox(
                label="อธิบายอาการฉุกเฉิน",
                placeholder="เช่น 'เจ็บหน้าอกรุนแรงและหายใจไม่ออก'",
                lines=3
            )
            emergency_output = gr.Markdown()

            triage_btn = gr.Button("⚠️ ประเมินอาการ", variant="stop")
            triage_btn.click(ai_agent.emergency_triage, inputs=[emergency_input], outputs=[emergency_output])

        with gr.Tab("ℹ️ เกี่ยวกับระบบ"):
            gr.Markdown("""
            ### ℹ️ เกี่ยวกับ ระบบผู้ช่วยทางการแพทย์อัจฉริยะสำหรับวิเคราะห์อาการเบื้องต้นและให้คำแนะนำทางการแพทย์ภาษาไทย พร้อม RAG

            **คุณสมบัติหลัก:**
            - 🤖 ผู้ช่วย AI สำหรับให้คำปรึกษาทางการแพทย์เบื้องต้น
            - 🏥 ระบบ RAG (Retrieval Augmented Generation) สำหรับข้อมูลทางการแพทย์
            - 📚 ฐานความรู้ทางการแพทย์ภาษาไทย
            - 🖼️ การวิเคราะห์ภาพทางการแพทย์เบื้องต้น
            - 🎤 การประมวลผลเสียงและคำพูด
            - 🚨 ระบบประเมินอาการฉุกเฉิน
            - 🔒 การปกป้องข้อมูลส่วนบุคคลตาม PDPA

            **ข้อมูลเทคนิค:**
            - **เวอร์ชัน:** 2.0
            - **อัปเดตล่าสุด:** สิงหาคม 2025
            - **ผู้พัฒนา:** บริษัท วี89 เทคโนโลยี จำกัด
            - **มาตรฐานทางการแพทย์:** HL7 FHIR R4
            - **การปกป้องข้อมูล:** PDPA Compliance

            **⚠️ ข้อจำกัดความรับผิดชอบ:**
            ระบบนี้เป็นเครื่องมือช่วยสนับสนุนการตัดสินใจทางการแพทย์เบื้องต้นเท่านั้น
            ไม่ใช่การแทนที่การวินิจฉัยของแพทย์ผู้เชี่ยวชาญ
            กรุณาปรึกษาแพทย์สำหรับการวินิจฉัยและการรักษาที่เหมาะสม
            """)

        # Footer
        gr.Markdown("""
        ---
        **พัฒนาโดย บริษัท วี89 เทคโนโลยี จำกัด** | 📧 ติดต่อ: support@v89.tech | 📞 โทร: 02-XXX-XXXX

        *ระบบนี้ผ่านการทดสอบเบื้องต้นและพร้อมให้บริการคำปรึกษาทางการแพทย์พื้นฐาน*
        """)

    return demo

# Create and launch the interface
print("🎨 Creating Gradio interface...")
demo = create_gradio_interface()

print("🚀 Launching the application...")
if __name__ == "__main__":
    demo.launch(
        server_name="0.0.0.0",
        #server_port=7860,
        share=True,
        debug=False
    )